# 🥈 Silver — Clean + NLP Features

Adds sentiment (VADER), text length features, engagement ratios.
All downstream analysis notebooks read from Silver.


In [2]:
# pip install vaderSentiment before running this notebook
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

spark = (
    SparkSession.builder
    .appName('Reddit-Silver')
    .master('local[2]')
    .config('spark.driver.memory', '4g')
    .config('spark.sql.shuffle.partitions', '8')
    .config('spark.sql.adaptive.enabled', 'true')
    .getOrCreate()
)
spark.sparkContext.setLogLevel('WARN')

#BRONZE_PATH = '../data/bronze/parquet'
BRONZE_PATH = '/mnt/c/Users/gusmc/OneDrive/Desktop/reddit_historical_data/data/bronze/parquet'

#SILVER_PATH = '../data/silver/posts'
SILVER_PATH = '/mnt/c/Users/gusmc/OneDrive/Desktop/reddit_historical_data/data/silver/posts'


bronze = spark.read.parquet(BRONZE_PATH)
print(f'Loaded {bronze.count():,} rows')

26/02/27 02:15:13 WARN Utils: Your hostname, terminator resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/27 02:15:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/27 02:15:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
[Stage 1:=======================================================> (43 + 1) / 44]

Loaded 9,341,343 rows


In [3]:
# VADER sentiment UDF
# UDF = User Defined Function — lets you run Python code row-by-row in Spark
# We broadcast the analyser so each executor gets one copy (not re-created per row)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Broadcast the analyser to all executors
analyser_broadcast = spark.sparkContext.broadcast(SentimentIntensityAnalyzer())

@F.udf(returnType=DoubleType())
def get_sentiment(text):
    if not text:
        return 0.0
    try:
        scores = analyser_broadcast.value.polarity_scores(text[:512])  # cap at 512 chars
        return float(scores['compound'])
    except:
        return 0.0

print('VADER UDF registered ✓')

VADER UDF registered ✓


In [4]:
# Build Silver — all derived columns
silver = (
    bronze
    .fillna({'score': 0, 'num_comments': 0, 'upvote_ratio': 0.0,
             'total_awards_received': 0})

    # Sentiment on title (fast) and full_text (richer but slower)
    .withColumn('title_sentiment',    get_sentiment('title'))
    .withColumn('fulltext_sentiment', get_sentiment('full_text'))

    # Sentiment label
    .withColumn('sentiment_label',
        F.when(F.col('title_sentiment') >=  0.05, 'positive')
         .when(F.col('title_sentiment') <= -0.05, 'negative')
         .otherwise('neutral')
    )

    # Text features
    .withColumn('title_word_count',
        F.size(F.split(F.trim('title'), r'\s+'))
    )
    .withColumn('selftext_word_count',
        F.when(
            F.col('selftext').isNotNull() & (F.col('selftext') != ''),
            F.size(F.split(F.trim('selftext'), r'\s+'))
        ).otherwise(0)
    )
    .withColumn('has_body', F.col('selftext_word_count') > 10)

    # Engagement features
    .withColumn('engagement_score',
        F.col('score') + (F.col('num_comments') * 2)
    )
    # Ragebait ratio: high comments relative to score = controversy
    .withColumn('controversy_ratio',
        F.when(F.col('score') > 0,
            F.col('num_comments').cast('double') / F.col('score').cast('double')
        ).otherwise(F.col('num_comments').cast('double'))
    )

    # Time features
    .withColumn('hour_of_day',  F.hour('created_ts'))
    .withColumn('day_of_week',  F.dayofweek('created_ts'))  # 1=Sunday, 7=Saturday
    .withColumn('week',         F.weekofyear('created_ts').cast('string'))
    .withColumn('year_month',   F.date_format('created_ts', 'yyyy-MM'))

    # Post age bucket at time of snapshot
    .withColumn('score_per_comment',
        F.when(F.col('num_comments') > 0,
            F.col('score').cast('double') / F.col('num_comments')
        ).otherwise(F.col('score').cast('double'))
    )

    # Is removed?
    .withColumn('is_removed',
        F.col('removed_by_category').isNotNull()
    )
)

silver.select('subreddit','title','title_sentiment','sentiment_label',
              'controversy_ratio','engagement_score').show(5, truncate=60)

+---------+------------------------------------------------------------+---------------+---------------+------------------+----------------+
|subreddit|                                                       title|title_sentiment|sentiment_label| controversy_ratio|engagement_score|
+---------+------------------------------------------------------------+---------------+---------------+------------------+----------------+
|    aitah|                     aitah for ruining my mother's birthday?|          -0.25|       negative|1.3333333333333333|              11|
|    aitah|                     Didn’t hire an older dog walker. AITAH?|            0.0|        neutral|0.9230769230769231|             111|
|    aitah|AITAH for Wanting to change what each groomsmen gets for ...|            0.0|        neutral|1.3333333333333333|              11|
|    aitah|AITAH for getting mad over my gf having emotional gifts f...|        -0.3818|       negative|               4.4|              49|
|    aitah|  

In [5]:
(
    silver.write
    .mode('overwrite')
    .partitionBy('subreddit', 'year')
    .parquet(SILVER_PATH)
)
print('Silver done ✓')
spark.stop()

26/02/27 02:17:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

Silver done ✓
